In [1]:
# Imports 
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import operator

In [2]:
# load the data, keep only selected columns
# Read the file using super fast Pandas.read_csv
def load_required_data(path, required_columns):
    dataframe = pd.read_json(path, lines=True)
    dataframe = dataframe[required_columns]
    return dataframe, dataframe.values

In [3]:
dataframe, values = load_required_data('../../Datasets/Apps Dataset/train_15000.json', ["asin", "reviewerID", "overall"])

In [4]:
dataframe[:10]

,asin,reviewerID,overall
0,B00ANT8OF6,A00100742Q4O8VH0YMUBZ,4
1,B00AMR1HZ8,A00100742Q4O8VH0YMUBZ,4
2,B004WGGQPQ,A00100742Q4O8VH0YMUBZ,5
3,B006C1ZSO4,A00100742Q4O8VH0YMUBZ,5
4,B008LY1B32,A00100742Q4O8VH0YMUBZ,4
5,B007KPT2N4,A00100742Q4O8VH0YMUBZ,1
6,B0080JJLBW,A00100742Q4O8VH0YMUBZ,5
7,B009P8EMCK,A00100742Q4O8VH0YMUBZ,5
8,B00I8Q77Y0,A001619027H9L9EG4UVRB,5
9,B00H0BGCJK,A001619027H9L9EG4UVRB,5


In [5]:
dataframe.shape

(15000, 3)

In [6]:
# Create user-item matrix given a matrix and mapping from original matrix columns to new matrix rows/columns
def create_user_item_matrix(data, rowMapping, columnMapping):
    # Create table of users to items by taking row and column pivots
    # Rows of user-item table are users
    rows, row_pos = np.unique(data[:, rowMapping], return_inverse=True)
    # Columns of user-item table are movies
    cols, col_pos = np.unique(data[:, columnMapping], return_inverse=True)

    pivot_table = np.zeros((len(rows), len(cols)), dtype=data.dtype)
    pivot_table[row_pos, col_pos] = data[:, 2]
    return pivot_table, rows, cols

In [7]:
ratings_matrix, rows, cols = create_user_item_matrix(values, rowMapping=1, columnMapping=0)
print (ratings_matrix.shape)

(3139L, 1842L)


In [8]:
pd.DataFrame(ratings_matrix[:10], index = rows[:10], columns= cols)

,B004ALVL6W,B004ANMWPY,B004DLNC4I,B004DLPXAO,B004DM1OAQ,B004DM1ZQY,B004DPBGCO,B004DPC5Y2,B004DPCSKI,B004DPIEF6,...,B00K7WGUKA,B00KFNXUY0,B00KGCNRAM,B00KI5Q8X0,B00KMX5V8G,B00KOEHQCW,B00KQHVWWC,B00KSOQ66K,B00KWVZ750,B00L3MNCNQ
A00100742Q4O8VH0YMUBZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A001619027H9L9EG4UVRB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A002359833QJM7OQHCXWY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0037670NPLI11RBWYFA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A003841815JTX0JFLR8B1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0038872349TB5N0JHQQW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00455683H6M1GQZMPQPV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0
A0047670XAAJD587LXS7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00501041JRAPWYLPQ4TE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00507662MEMHI1YMGQ15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
def get_index_map(arr):
    index_map = {}
    for el in arr:
        index_map[el] = len(index_map)
    return index_map

In [10]:
# Get unique movies and users
unique_items = np.unique(values[:,0])
unique_users = np.unique(values[:,1])

In [11]:
unique_items.shape

(1842L,)

In [12]:
unique_users.shape

(3139L,)

In [13]:
mean_overall = np.true_divide(ratings_matrix.sum(),(ratings_matrix!=0).sum())

In [14]:
def get_users_who_rated_item(item_id):
    return np.where(ratings_matrix[:,item_idx_map[item_id]] > 0)[0]

def get_users_who_rated_item_by_index(item_idx):
    return np.where(ratings_matrix[:,item_idx] > 0)

def predict(user_id, item_id):
    return mean_overall

In [16]:
test_dataframe, test_values = load_required_data('../../Datasets/Apps Dataset/test_5000.json', ["asin", "reviewerID", "overall"])

In [17]:
test_dataframe['reviewerID'].unique().shape

(2507L,)

In [18]:
# Returns DF containing predictions, MAE, RMSE
def get_predictions(dataframe):
    df = dataframe.copy()
    print ("Predicting ratings..")
    start_time = time.time()
    df['Prediction'] = df.apply(lambda row: predict_row(row) , axis=1)
    MAE = round(mean_absolute_error(df['overall'], df['Prediction']), 2)
    RMSE = round(np.sqrt(mean_squared_error(df['overall'], df['Prediction'])), 2)
    print "Done Predicting in",  round((time.time() - start_time), 2), "seconds"
    return df, MAE, RMSE

i = 0
def predict_row(row):
    global i
    i+= 1
    prediction = predict(row['reviewerID'], row['asin'])
    print ("#", i, "Actual = ", row['overall'], "Predictioned = ", prediction)
    return prediction

In [19]:
df, MAE, RMSE = get_predictions(test_dataframe)

Predicting ratings..
('#', 1, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 2, 'Actual = ', 2L, 'Predictioned = ', 4.0581333333333331)
('#', 3, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 4, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 5, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 6, 'Actual = ', 3L, 'Predictioned = ', 4.0581333333333331)
('#', 7, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 8, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 9, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 10, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 11, 'Actual = ', 1L, 'Predictioned = ', 4.0581333333333331)
('#', 12, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 13, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 14, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 15, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331

('#', 989, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 990, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 991, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 992, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 993, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 994, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 995, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 996, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 997, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 998, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 999, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 1000, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 1001, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 1002, 'Actual = ', 1L, 'Predictioned = ', 4.0581333333333331)
('#', 1003, 'Actual = ', 4L, 'Predictioned = ', 4.058133333

('#', 2770, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 2771, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 2772, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 2773, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 2774, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 2775, 'Actual = ', 3L, 'Predictioned = ', 4.0581333333333331)
('#', 2776, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 2777, 'Actual = ', 3L, 'Predictioned = ', 4.0581333333333331)
('#', 2778, 'Actual = ', 3L, 'Predictioned = ', 4.0581333333333331)
('#', 2779, 'Actual = ', 3L, 'Predictioned = ', 4.0581333333333331)
('#', 2780, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 2781, 'Actual = ', 3L, 'Predictioned = ', 4.0581333333333331)
('#', 2782, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 2783, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 2784, 'Actual = ', 5L, 'Predictioned = ', 

('#', 4713, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 4714, 'Actual = ', 2L, 'Predictioned = ', 4.0581333333333331)
('#', 4715, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 4716, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 4717, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 4718, 'Actual = ', 2L, 'Predictioned = ', 4.0581333333333331)
('#', 4719, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 4720, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 4721, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 4722, 'Actual = ', 1L, 'Predictioned = ', 4.0581333333333331)
('#', 4723, 'Actual = ', 4L, 'Predictioned = ', 4.0581333333333331)
('#', 4724, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 4725, 'Actual = ', 1L, 'Predictioned = ', 4.0581333333333331)
('#', 4726, 'Actual = ', 5L, 'Predictioned = ', 4.0581333333333331)
('#', 4727, 'Actual = ', 4L, 'Predictioned = ', 

In [20]:
print "MAE=", MAE
print "RMSE=", RMSE

MAE= 0.98
RMSE= 1.26


In [21]:
sparse_ratings = csr_matrix(ratings_matrix, dtype='float')

In [22]:
model_knn = NearestNeighbors(metric = 'cosine', n_neighbors = 6)
model_knn.fit(sparse_ratings)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=6, p=2, radius=1.0)

In [23]:
def get_recommendations(user_index, n_reccomendations=5, n_neighbors=10):
    distances,indices = model_knn.kneighbors(ratings_matrix[user_index].reshape(1, -1), n_neighbors=n_neighbors)
    probable_items = list()
    for user in indices[0]:
        if user == user_index:
            #print "Recommendations for user ", unique_users[user_index]
            continue
        # all other users
        for items in np.nonzero(ratings_matrix[user]):
            for item in items:
                if ratings_matrix[user_index][item] == 0:
                    probable_items.append(unique_items[item])

    predictions = {}
    for item in probable_items:
        predictions[item] = predict(unique_users[0], item)
        sorted_predictions = sorted(predictions.items(), key=operator.itemgetter(1))
    return [i[0] for i in sorted_predictions[:n_reccomendations]]

In [24]:
def get_user_items_rated_df(user_id):
    df = pd.DataFrame()
    np.nonzero(ratings_matrix[user_id])
    for i in np.nonzero(ratings_matrix[user_id]):
        x = pd.DataFrame({"item":unique_items[i], "rating":ratings_matrix[user_id][i]})
        df = df.append(x)
        return df

def print_items_rated(user_id):
    np.nonzero(ratings_matrix[user_id])
    for i in unique_items[np.nonzero(ratings_matrix[user_id])]:
        print i

In [25]:
get_user_items_rated_df(0)

,item,rating
0,B004WGGQPQ,5
1,B006C1ZSO4,5
2,B007KPT2N4,1
3,B0080JJLBW,5
4,B008LY1B32,4
5,B009P8EMCK,5
6,B00AMR1HZ8,4
7,B00ANT8OF6,4


In [26]:
#print get_recommendations(user_index=0)
import sys
sys.path.insert(0, '../Helpers and Scripts/')
import asin_to_rec

asin_to_rec.get_recommendation_items('B00A13BI7U')